# DB2-Salesforce connector: Organization information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
object_id = 'organization__c'
external_id = 'Domain__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",


    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain organization information from DB2 

In [222]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/wang159_myrmekes' \
                                               %(sql_login_params['username'], sql_login_params['password']))

In [238]:
# isolate data
i_char_df = pd.read_sql_query('select * from institution_characteristics', engine)
i_msi_df = pd.read_sql_query('select * from MSI_status',engine)
i_cls_df = pd.read_sql_query('select * from institution_classification', engine)
i_cls_label_df = pd.read_sql_query('select * from institution_classification_labels', engine)
i_cls_var_df = pd.read_sql_query('select * from institution_variable_labels', engine)
usn_ranks_df = pd.read_sql_query('select * from us_news_rankings',engine)

## Getting institution domain information

In [239]:
# get domain address from URL
import re

def get_domain(this_domain_name):

    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z-]+", ".", this_domain_name.lower())
    name_list = this_domain_name.split('.')

    # index of 'edu'
    try:
        edu_index = name_list.index('edu')
    except:
        return None
    
    if edu_index > 0:
        return name_list[edu_index-1]+'.edu'
    else:
        return None

    

unitid_url_df = i_char_df[['WEBADDR', 'UNITID']].copy()

unitid_url_df.loc[:, 'domain'] = unitid_url_df.WEBADDR.apply(get_domain)

# drop NaN rows
unitid_url_df.dropna(inplace=True)

In [240]:
display(unitid_url_df.head(1))
display(usn_ranks_df.head(1))

,WEBADDR,UNITID,domain
0,www.aamu.edu/,100654,aamu.edu


,index,UNITID,INSTNM,WEBADDR,usn_gr_ae,usn_gr_EE,chem_gr_anal,chem_gr_inorg,chem_gr_phy,chem_gr_bio,chem_gr_org,chem_gr_theo,phys_gr_atom,phys_gr_quan,usn_ug_CE_w_doct,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_ug_eng_no_doctorate,usn_gr_IE,usn_gr_mse,usn_gr_me,usn_gr_ce,usn_ug_eng_w_doct,usn_ug_EE_no_doct,usn_ug_EE_w_doct,phys_gr_con,usn_gr_eng,usn_natl_publ,usn_natl,Unnamed: 33
0,0,186131,Princeton University,www.princeton.edu,9.0,9.0,NaN,13.0,11.0,None,7,12.0,NaN,10.0,11.0,13.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,8.0,12.0,NaN,13.0,8.0,21,NaN,1,None


In [241]:
## filtering institutional classification dataframe for 
## i_cls_df -> keep all because i_cls_labels_df may need them

usn_ranks_df = usn_ranks_df.drop(columns=['INSTNM','WEBADDR'])

In [242]:
# join domain into institution classification table by unitid
df = pd.merge(i_cls_df, unitid_url_df, how='inner', left_on='UNITID', right_on='UNITID')

# display
df.head(1)

,index,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,IPUG2018,IPGRAD2018,ENRPROFILE2018,UGPROFILE2018,SIZESET2018,CCE2015,OBEREG,SECTOR,ICLEVEL,CONTROL,LOCALE,LANDGRNT,MEDICAL,HBCU,TRIBAL,HSI,MSI,WOMENS,COPLAC,CUSU,CUMU,ASSOCDEG,BACCDEG,MASTDEG,DOCRSDEG,DOCPPDEG,DOCOTHDEG,TOTDEG,S&ER&D,NONS&ER&D,PDNFRSTAFF,FACNUM,HUM_RSD,SOCSC_RSD,STEM_RSD,OTHER_RSD,DRSA&S,DRSPROF,OGRDA&S,OGRDPROF,A&SBADEG,PROFBADEG,ASC1C2TRNS,ASC1C2CRTC,FALLENR16,ANENR1617,FALLENR17,FALLFTE17,UGTENR17,GRTENR17,UGDSFTF17,UGDSPTF17,UGNDFT17,UGNDPT17,GRFTF17,GRPTF17,UGN1STTMFT17,UGN1STTMPT17,UGNTRFT17,UGNTRPT17,FAITHFLAG,OTHSFFLAG,NUMCIP2,LRGSTCIP2,PCTLRGST,UGCIP4PR,GRCIP4PR,COEXPR,PCTCOEX,DOCRESFLAG,MAXGPEDUC,MAXGPBUS,MAXGPOTH,NGCIP2PXDR,NGCIP2DR,ROOMS,ACTCAT,NSAT,NACT,NSATACT,SATV25,SATM25,SATCMB25,SATACTEQ25,ACTCMP25,ACTFINAL,Unnamed95,Unnamed96,WEBADDR,domain
0,0,177834,A T Still University of Health Sciences,Kirksville,MO,52,25,25,25,25,0,18,7,0,18,0,4,2,1,2,33,2,1,2,2,0,0,0,0,0,0,0,0,283,102,661,0,1046,None,None,None,None,None,None,None,None,0,102,13,931,0,0,0,0,3569.0,4303.0,3723,3085.0,0,3723,0.0,0.0,0.0,0.0,2766.0,957.0,0.0,0.0,0.0,0.0,0,1,3,51,0.966539,0,10,0,0.0,0,0,0,1,2,2,120,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,None,NaN,WWW.ATSU.EDU,atsu.edu


In [243]:
## factoring in information from us news and world report
#cols_to_use = ['UNITID','NAME','CITY','STABBR','BASIC2018','domain'] t_df = t_df.drop(columns=['index'])
t_df = pd.merge(df,usn_ranks_df,left_on='UNITID',right_on='UNITID') 

display(t_df.head(1))

,index_x,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,IPUG2018,IPGRAD2018,ENRPROFILE2018,UGPROFILE2018,SIZESET2018,CCE2015,OBEREG,SECTOR,ICLEVEL,CONTROL,LOCALE,LANDGRNT,MEDICAL,HBCU,TRIBAL,HSI,MSI,WOMENS,COPLAC,CUSU,CUMU,ASSOCDEG,BACCDEG,MASTDEG,DOCRSDEG,DOCPPDEG,DOCOTHDEG,TOTDEG,S&ER&D,NONS&ER&D,PDNFRSTAFF,FACNUM,HUM_RSD,SOCSC_RSD,STEM_RSD,OTHER_RSD,DRSA&S,DRSPROF,OGRDA&S,OGRDPROF,A&SBADEG,PROFBADEG,ASC1C2TRNS,ASC1C2CRTC,FALLENR16,ANENR1617,FALLENR17,FALLFTE17,UGTENR17,GRTENR17,UGDSFTF17,UGDSPTF17,UGNDFT17,UGNDPT17,GRFTF17,GRPTF17,UGN1STTMFT17,UGN1STTMPT17,UGNTRFT17,UGNTRPT17,FAITHFLAG,OTHSFFLAG,NUMCIP2,LRGSTCIP2,PCTLRGST,UGCIP4PR,GRCIP4PR,COEXPR,PCTCOEX,DOCRESFLAG,MAXGPEDUC,MAXGPBUS,MAXGPOTH,NGCIP2PXDR,NGCIP2DR,ROOMS,ACTCAT,NSAT,NACT,NSATACT,SATV25,SATM25,SATCMB25,SATACTEQ25,ACTCMP25,ACTFINAL,Unnamed95,Unnamed96,WEBADDR,domain,index_y,usn_gr_ae,usn_gr_EE,chem_gr_anal,chem_gr_inorg,chem_gr_phy,chem_gr_bio,chem_gr_org,chem_gr_theo,phys_gr_atom,phys_gr_quan,usn_ug_CE_w_doct,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_ug_eng_no_doctorate,usn_gr_IE,usn_gr_mse,usn_gr_me,usn_gr_ce,usn_ug_eng_w_doct,usn_ug_EE_no_doct,usn_ug_EE_w_doct,phys_gr_con,usn_gr_eng,usn_natl_publ,usn_natl,Unnamed: 33
0,0,177834,A T Still University of Health Sciences,Kirksville,MO,52,25,25,25,25,0,18,7,0,18,0,4,2,1,2,33,2,1,2,2,0,0,0,0,0,0,0,0,283,102,661,0,1046,None,None,None,None,None,None,None,None,0,102,13,931,0,0,0,0,3569.0,4303.0,3723,3085.0,0,3723,0.0,0.0,0.0,0.0,2766.0,957.0,0.0,0.0,0.0,0.0,0,1,3,51,0.966539,0,10,0,0.0,0,0,0,1,2,2,120,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,None,NaN,WWW.ATSU.EDU,atsu.edu,2017,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None


In [244]:
display(i_cls_label_df.head(1))
display(i_cls_label_df.groupby(by=['Variable']))
display(i_cls_label_df['Variable'].head(600))

,index,Variable,Value,Label
0,0,CC2000,-3.0,"Not classified, not in classification universe"


0          CC2000
1          CC2000
2          CC2000
3          CC2000
4          CC2000
          ...    
265    DOCRESFLAG
266    DOCRESFLAG
267        ACTCAT
268        ACTCAT
269        ACTCAT
Name: Variable, Length: 270, dtype: object

In [245]:
def replace_with_label(this_df, c_df):
    #print(this_df.head(2))
    #print(c_df.head(2))
    
    #this_df = this_df['Variable'].to_frame()
    #print(this_df.head(2))
    
    this_var = this_df.Variable.iloc[0].strip()
    #this_var = this_df.iloc[0].strip()
    this_v_l = this_df[['Value', 'Label']]
    
    this_label = pd.merge(c_df[[this_var]], this_v_l, how='left', left_on=this_var, right_on='Value')
    c_df.loc[:, this_var] = this_label['Label']
    
    
i_cls_label_df.groupby(by=['Variable']).apply(lambda x: replace_with_label(x, t_df))

""


In [246]:
# display
i_cls_label_df.head(2)

,index,Variable,Value,Label
0,0,CC2000,-3.0,"Not classified, not in classification universe"
1,1,CC2000,15.0,Doctoral/Research Universities—Extensive


In [247]:
# On Salesforce side, domain is unique indentifier of each organization.
# For multiple institutions with same domain (univ. with satellite campuses), use entry with highest total enrollment (FALLENR17)

unique_domain_df = t_df.sort_values(by='FALLENR17', ascending=False).groupby('domain').head(1)

In [248]:
display(unique_domain_df.head(2))
print(unique_domain_df.loc[:,'NAME'])

,index_x,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,IPUG2018,IPGRAD2018,ENRPROFILE2018,UGPROFILE2018,SIZESET2018,CCE2015,OBEREG,SECTOR,ICLEVEL,CONTROL,LOCALE,LANDGRNT,MEDICAL,HBCU,TRIBAL,HSI,MSI,WOMENS,COPLAC,CUSU,CUMU,ASSOCDEG,BACCDEG,MASTDEG,DOCRSDEG,DOCPPDEG,DOCOTHDEG,TOTDEG,S&ER&D,NONS&ER&D,PDNFRSTAFF,FACNUM,HUM_RSD,SOCSC_RSD,STEM_RSD,OTHER_RSD,DRSA&S,DRSPROF,OGRDA&S,OGRDPROF,A&SBADEG,PROFBADEG,ASC1C2TRNS,ASC1C2CRTC,FALLENR16,ANENR1617,FALLENR17,FALLFTE17,UGTENR17,GRTENR17,UGDSFTF17,UGDSPTF17,UGNDFT17,UGNDPT17,GRFTF17,GRPTF17,UGN1STTMFT17,UGN1STTMPT17,UGNTRFT17,UGNTRPT17,FAITHFLAG,OTHSFFLAG,NUMCIP2,LRGSTCIP2,PCTLRGST,UGCIP4PR,GRCIP4PR,COEXPR,PCTCOEX,DOCRESFLAG,MAXGPEDUC,MAXGPBUS,MAXGPOTH,NGCIP2PXDR,NGCIP2DR,ROOMS,ACTCAT,NSAT,NACT,NSATACT,SATV25,SATM25,SATCMB25,SATACTEQ25,ACTCMP25,ACTFINAL,Unnamed95,Unnamed96,WEBADDR,domain,index_y,usn_gr_ae,usn_gr_EE,chem_gr_anal,chem_gr_inorg,chem_gr_phy,chem_gr_bio,chem_gr_org,chem_gr_theo,phys_gr_atom,phys_gr_quan,usn_ug_CE_w_doct,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_ug_eng_no_doctorate,usn_gr_IE,usn_gr_mse,usn_gr_me,usn_gr_ce,usn_ug_eng_w_doct,usn_ug_EE_no_doct,usn_ug_EE_w_doct,phys_gr_con,usn_gr_eng,usn_natl_publ,usn_natl,Unnamed: 33
3687,3849,484613,University of Phoenix-Arizona,Tempe,AZ,"Not classified, not in classification universe","Not classified, not in classification universe",NaN,Doctoral Universities: Moderate Research Activity,Doctoral/Professional Universities,"Professions focus, some graduate coexistence",Research Doctoral: Professional-dominant,High undergraduate,"Four-year, full-time, inclusive, higher transf...","Four-year, large, primarily nonresidential",Not Classified,Southwest AZ NM OK TX,"Private for-profit, 4-year or above",Four or more years,Private for-profit,City Midsize,No,No,No,No,No,No,No,No,No,No,7290,14254,7239,354,0,0,29137,None,None,None,None,None,None,None,None,11,343,820,6419,2757,11497,9387,4588,131629.0,190750.0,103975,103975.0,80898,23077,77269.0,0.0,3629.0,0.0,23077.0,0.0,4209.0,0.0,5075.0,0.0,Not a special focus faith-related institution,Not a special focus institution,15,52,0.400625,29,21,11,0.379310,Conferred research/scholarship doctoral degrees,0,1,0,8,4,0,Inclusive,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,None,NaN,www.phoenix.edu,phoenix.edu,6340,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None
3996,4171,433387,Western Governors University,Salt Lake City,UT,"Not classified, not in classification universe",NaN,Master's L: Master's Colleges and Universities...,Master's Colleges & Universities: Larger Programs,Master's Colleges & Universities: Larger Programs,"Professions focus, high graduate coexistence","Postbaccalaureate: Business-dominant, with Art...",Majority undergraduate,"Four-year, full-time, inclusive, higher transf...","Four-year, large, primarily nonresidential",Not Classified,Rocky Mountains CO ID MT UT WY,"Private not-for-profit, 4-year or above",Four or more years,Private not-for-profit,Suburb Large,No,No,No,No,No,No,No,No,No,No,0,13087,7913,0,0,0,21000,None,None,None,None,None,None,None,None,0,0,106,7807,487,12600,0,0,84289.0,121017.0,98627,98627.0,72385,26242,72385.0,0.0,0.0,0.0,26242.0,0.0,0.0,0.0,71356.0,0.0,Not a special focus faith-related institution,Not a special focus institution,4,51,0.457857,18,17,13,0.722222,Did not confer research/scholarship doctoral d...,0,1,0,4,0,0,Inclusive,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,None,NaN,www.wgu.edu,wgu.edu,4771,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None


3687                        University of Phoenix-Arizona
3996                         Western Governors University
3095                    Southern New Hampshire University
1399                              Grand Canyon University
1653                           Ivy Tech Community College
                              ...                        
501                          California Christian College
2160    Monteclaro Escuela de Hoteleria y Artes Culina...
1881                Los Angeles Academy of Figurative Art
3411    Theological Seminary of the Reformed Episcopal...
1451                         Harrington College of Design
Name: NAME, Length: 3342, dtype: object


In [249]:
## adding the MSI breakdown information
i_msi_df = i_msi_df.iloc[1:,1:3]

# split i_msi into columns with Yes or No indicators
# have MSI, HSI, HCBU, PBI, TCU, NASNTI, ANNH, AANAPISI
i_msi_df['MSI'] = 'Yes'
i_msi_df['MSI2'] = i_msi_df['2019 List of Minority Serving Institutions (MSIs)']
i_msi_df = i_msi_df.drop(columns=['2019 List of Minority Serving Institutions (MSIs)'])

i_msi_df['Name'] = i_msi_df['Unnamed: 1']
i_msi_df = i_msi_df.drop(columns=['Unnamed: 1'])


display(i_msi_df.head(1))


,MSI,MSI2,Name
1,Yes,TCU,Aaniiih Nakoda College


In [250]:
## extract msi subcategories
msi_cats = i_msi_df['MSI2'].apply(lambda x: x.split()).apply(lambda x: [i for i in x if i != '&'])

for index,vals in enumerate(msi_cats):
    for item in vals:
        i_msi_df.loc[index,item] = 'Yes'

In [251]:
msi_df = i_msi_df.drop(columns=['MSI2'])
display(msi_df.head(2))

,MSI,Name,TCU,HSI,AANAPISI,HBCU,ANNH,NASNTI,PBI
1,Yes,Aaniiih Nakoda College,NaN,Yes,NaN,NaN,NaN,NaN,NaN
2,Yes,Adams State University,NaN,NaN,Yes,NaN,NaN,NaN,NaN


In [252]:
# add msi information onto the unique_domain_df
unique_domain_df = unique_domain_df.drop(columns=['HBCU','MSI','HSI'])

unique_domain_df = unique_domain_df.merge(msi_df,left_on='NAME',right_on='Name')
unique_domain_df = unique_domain_df.drop(columns=['Name'])
display(unique_domain_df.head(2))

,index_x,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,IPUG2018,IPGRAD2018,ENRPROFILE2018,UGPROFILE2018,SIZESET2018,CCE2015,OBEREG,SECTOR,ICLEVEL,CONTROL,LOCALE,LANDGRNT,MEDICAL,TRIBAL,WOMENS,COPLAC,CUSU,CUMU,ASSOCDEG,BACCDEG,MASTDEG,DOCRSDEG,DOCPPDEG,DOCOTHDEG,TOTDEG,S&ER&D,NONS&ER&D,PDNFRSTAFF,FACNUM,HUM_RSD,SOCSC_RSD,STEM_RSD,OTHER_RSD,DRSA&S,DRSPROF,OGRDA&S,OGRDPROF,A&SBADEG,PROFBADEG,ASC1C2TRNS,ASC1C2CRTC,FALLENR16,ANENR1617,FALLENR17,FALLFTE17,UGTENR17,GRTENR17,UGDSFTF17,UGDSPTF17,UGNDFT17,UGNDPT17,GRFTF17,GRPTF17,UGN1STTMFT17,UGN1STTMPT17,UGNTRFT17,UGNTRPT17,FAITHFLAG,OTHSFFLAG,NUMCIP2,LRGSTCIP2,PCTLRGST,UGCIP4PR,GRCIP4PR,COEXPR,PCTCOEX,DOCRESFLAG,MAXGPEDUC,MAXGPBUS,MAXGPOTH,NGCIP2PXDR,NGCIP2DR,ROOMS,ACTCAT,NSAT,NACT,NSATACT,SATV25,SATM25,SATCMB25,SATACTEQ25,ACTCMP25,ACTFINAL,Unnamed95,Unnamed96,WEBADDR,domain,index_y,usn_gr_ae,usn_gr_EE,chem_gr_anal,chem_gr_inorg,chem_gr_phy,chem_gr_bio,chem_gr_org,chem_gr_theo,phys_gr_atom,phys_gr_quan,usn_ug_CE_w_doct,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_ug_eng_no_doctorate,usn_gr_IE,usn_gr_mse,usn_gr_me,usn_gr_ce,usn_ug_eng_w_doct,usn_ug_EE_no_doct,usn_ug_EE_w_doct,phys_gr_con,usn_gr_eng,usn_natl_publ,usn_natl,Unnamed: 33,MSI,TCU,HSI,AANAPISI,HBCU,ANNH,NASNTI,PBI
0,1948,227182,Lone Star College System,The Woodlands,TX,Associate's Colleges,NaN,Assoc/Pub-S-MC: Associate's--Public Suburban-s...,Associate's Colleges: High Transfer-Mixed Trad...,Associate's Colleges: High Transfer-Mixed Trad...,Associate's Colleges: High Transfer,Not classified (Exclusively Undergraduate),Exclusively undergraduate two-year,"Two-year, higher part-time","Two-year, very large",Not Classified,Southwest AZ NM OK TX,"Public, 2-year",At least 2 but less than 4 years,Public,City Small,No,No,No,No,No,No,No,7223,0,0,0,0,0,7223,None,None,None,None,None,None,None,None,0,0,0,0,0,0,7710,1825,71551.0,110542.0,72336,37800.000000,72336,0,19618.0,39489.0,914.0,12315.0,0.0,0.0,6486.0,5888.0,2075.0,3229.0,Not a special focus faith-related institution,Not a special focus institution,19,24,0.746227,37,0,0,0.0,Did not confer research/scholarship doctoral d...,0,0,0,0,0,0,Inclusive,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000000,None,NaN,www.lonestar.edu,lonestar.edu,3541,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,Yes,NaN,Yes,Yes,NaN,NaN,NaN,NaN
1,3721,132903,University of Central Florida,Orlando,FL,Doctoral/Research Universities—Intensive,NaN,RU/VH: Research Universities (very high resear...,Doctoral Universities: Highest Research Activity,Doctoral Universities: Very High Research Acti...,"Professions plus arts & sciences, high graduat...","Research Doctoral: Comprehensive programs, wit...",High undergraduate,"Four-year, medium full-time , selective, highe...","Four-year, large, primarily nonresidential",Classified,Southeast AL AR FL GA KY LA MS NC SC TN VA WV,"Public, 4-year or above",Four or more years,Public,Suburb Large,No,Yes,No,No,No,Yes,No,457,13210,2197,279,150,0,16293,186009,47693,92,965,6,29,127,117,162,117,692,1655,4793,8417,457,0,64088.0,75824.0,66059,51728.333333,56853,9206,39685.0,16681.0,277.0,210.0,4601.0,4605.0,6678.0,192.0,4715.0,3136.0,Not a special focus faith-related institution,Not a special focus institution,25,52,0.190020,60,63,42,0.7,Conferred research/scholarship doctoral degrees,1,0,0,21,13,11641,More selective,4099,2780,6879,580.0,570.0,1150.0,23.0,24.0,23.404129,None,NaN,www.ucf.edu/,ucf.edu,133,53.0,58.0,NaN,NaN,NaN,None,None,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,70.0,61.0,83.0,NaN,NaN,NaN,74,79.0,None,None,Yes,NaN,NaN,Yes,NaN,NaN,NaN,NaN


## Obtain organization metadata from Salesforce

In [253]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
org_metadata = db_s.get_obj_metadata('Organization__c')

Obtained Salesforce access token ...... True


In [254]:
import json

# get all fields
field_list = json.loads(org_metadata)['fields']

# list of available field API names
available_fields = list()

for this_field in field_list:
    this_name = this_field['name']
    
    if len(this_name) > 3:
        if this_name[-3:] == '__c':
            # this is a custom-made field
            available_fields.append(this_name[:-3])
            
# display
display(available_fields)

['Domain',
 'Country',
 'Industry',
 'Size_Range',
 'MSI',
 'is_US_institution',
 'HBCU',
 'TCU',
 'BASIC2018',
 'usn_gr_eng',
 'usn_gr_EE',
 'usn_gr_mse',
 'usn_gr_ce',
 'usn_gr_me',
 'usn_gr_ae',
 'usn_gr_IE',
 'chem_gr_anal',
 'chem_gr_inorg',
 'chem_gr_phy',
 'chem_gr_bio',
 'chem_gr_org',
 'chem_gr_theo',
 'phys_gr_atom',
 'phys_gr_con',
 'phys_gr_quan',
 'usn_ug_eng_w_doct',
 'usn_ug_eng_no_doctorate',
 'usn_ug_EE_w_doct',
 'usn_ug_EE_no_doct',
 'usn_ug_CE_w_doct',
 'usn_ug_CE_no_doct',
 'usn_ug_ME_w_doct',
 'usn_ug_ME_no_doct',
 'usn_ug_ChE_w_doct',
 'usn_ug_ChE_no_doct',
 'usn_ug_IE_w_doct',
 'usn_ug_IE_no_doct',
 'usn_ug_MSE_w_doct',
 'usn_natl',
 'usn_natl_publ',
 'usn_ug_MSE_no_doct',
 'ANNH',
 'AANAPISI',
 'PBI',
 'HSI',
 'NASNTI']

## Match data with Salesforce format

In [255]:
# find columns that have matched fields in Salesforce
matched_columns = set(unique_domain_df.columns).intersection(set(available_fields))

# display
display(matched_columns)

{'AANAPISI',
 'ANNH',
 'BASIC2018',
 'HBCU',
 'HSI',
 'MSI',
 'NASNTI',
 'PBI',
 'TCU',
 'chem_gr_anal',
 'chem_gr_bio',
 'chem_gr_inorg',
 'chem_gr_org',
 'chem_gr_phy',
 'chem_gr_theo',
 'phys_gr_atom',
 'phys_gr_con',
 'phys_gr_quan',
 'usn_gr_EE',
 'usn_gr_IE',
 'usn_gr_ae',
 'usn_gr_ce',
 'usn_gr_eng',
 'usn_gr_me',
 'usn_gr_mse',
 'usn_natl',
 'usn_natl_publ',
 'usn_ug_CE_w_doct',
 'usn_ug_ChE_no_doct',
 'usn_ug_ChE_w_doct',
 'usn_ug_EE_no_doct',
 'usn_ug_EE_w_doct',
 'usn_ug_IE_no_doct',
 'usn_ug_IE_w_doct',
 'usn_ug_ME_no_doct',
 'usn_ug_ME_w_doct',
 'usn_ug_MSE_w_doct',
 'usn_ug_eng_no_doctorate',
 'usn_ug_eng_w_doct'}

In [256]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Domain__c'] = unique_domain_df['domain']

df_sf['is_US_institution__c'] = True
df_sf['Name'] = unique_domain_df['NAME']

# Transfer all matched columns to df_sf
for this_col in matched_columns:
    df_sf[this_col+'__c'] = unique_domain_df[this_col]

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Domain__c,lonestar.edu,ucf.edu
is_US_institution__c,True,True
Name,Lone Star College System,University of Central Florida
usn_ug_ME_w_doct__c,NaN,NaN
TCU__c,NaN,NaN
usn_ug_CE_w_doct__c,NaN,NaN
usn_gr_ae__c,NaN,53
chem_gr_theo__c,NaN,NaN
usn_ug_eng_no_doctorate__c,NaN,NaN
usn_gr_me__c,NaN,70


## To Salesforce Sales Cloud CRM

In [257]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [258]:
display(df_sf.head(2))

,Domain__c,is_US_institution__c,Name,usn_ug_ME_w_doct__c,TCU__c,usn_ug_CE_w_doct__c,usn_gr_ae__c,chem_gr_theo__c,usn_ug_eng_no_doctorate__c,usn_gr_me__c,phys_gr_atom__c,chem_gr_phy__c,HSI__c,usn_gr_ce__c,phys_gr_con__c,BASIC2018__c,usn_ug_ME_no_doct__c,PBI__c,chem_gr_anal__c,usn_natl_publ__c,ANNH__c,usn_ug_ChE_no_doct__c,usn_ug_IE_w_doct__c,usn_ug_ChE_w_doct__c,MSI__c,chem_gr_bio__c,chem_gr_org__c,usn_ug_EE_no_doct__c,usn_ug_eng_w_doct__c,usn_ug_EE_w_doct__c,usn_ug_MSE_w_doct__c,HBCU__c,usn_gr_IE__c,usn_gr_eng__c,phys_gr_quan__c,chem_gr_inorg__c,usn_natl__c,usn_gr_mse__c,AANAPISI__c,NASNTI__c,usn_ug_IE_no_doct__c,usn_gr_EE__c
0,lonestar.edu,True,Lone Star College System,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Associate's Colleges: High Transfer-Mixed Trad...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,Yes,NaN,NaN,NaN
1,ucf.edu,True,University of Central Florida,NaN,NaN,NaN,53.0,NaN,NaN,70.0,12.0,NaN,NaN,61.0,NaN,Doctoral Universities: Very High Research Acti...,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,Yes,None,None,NaN,83.0,NaN,NaN,NaN,NaN,74,NaN,NaN,None,54.0,Yes,NaN,NaN,58.0


In [259]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000OpVXmAAN
hello
[Success] CSV upload successful. Job ID = 7505w00000OpVXmAAN
[Success] Closing job successful. Job ID = 7505w00000OpVXmAAN


In [267]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000OpVXmAAN',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-19T02:37:03.000+0000',
 'systemModstamp': '2020-08-19T02:37:28.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 526,
 'numberRecordsFailed': 22,
 'retries': 0,
 'totalProcessingTime': 1927,
 'apiActiveProcessingTime': 1409,
 'apexProcessingTime': 0}

In [268]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",AANAPISI__c,ANNH__c,BASIC2018__c,Domain__c,HBCU__c,HSI__c,MSI__c,NASNTI__c,Name,PBI__c,TCU__c,chem_gr_anal__c,chem_gr_bio__c,chem_gr_inorg__c,chem_gr_org__c,chem_gr_phy__c,chem_gr_theo__c,is_US_institution__c,phys_gr_atom__c,phys_gr_con__c,phys_gr_quan__c,usn_gr_EE__c,usn_gr_IE__c,usn_gr_ae__c,usn_gr_ce__c,usn_gr_eng__c,usn_gr_me__c,usn_gr_mse__c,usn_natl__c,usn_natl_publ__c,usn_ug_CE_w_doct__c,usn_ug_ChE_no_doct__c,usn_ug_ChE_w_doct__c,usn_ug_EE_no_doct__c,usn_ug_EE_w_doct__c,usn_ug_IE_no_doct__c,usn_ug_IE_w_doct__c,usn_ug_ME_no_doct__c,usn_ug_ME_w_doct__c,usn_ug_MSE_w_doct__c,usn_ug_eng_no_doctorate__c,usn_ug_eng_w_doct__c\n'
 '"","DUPLICATE_VALUE:Duplicate external id specified: gccaz.edu:Domain__c '
 '--","","","Associate\'s Colleges: High Transfer-Mixed '
 'Traditional/Nontraditional","gccaz.edu","","Yes","Yes","","Glendale '
 'Community '
 'College","","","","","","","","","true","","","","","","","","","","","","","","","","","","","","","","","",""\n'
 '"